In [35]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [36]:
path ='/Users/fyenne/Downloads/booooks/semester5/R_ML/Mar_NCAA/sb'

In [37]:
tourney_result = pd.read_csv(path+'/train_m_stg2.csv')
test_df = pd.read_csv(path+'/test_m_stg2.csv')

In [38]:
# train_list=list(tourney_result.columns)[:-1]
# tourney_result.columns[-1]

In [39]:
tourney_result = tourney_result.rename(columns = {'train_label_my':'result'})

In [40]:
import h2o
h2o.init(
  nthreads=-1,            ## -1: use all available threads
  max_mem_size = "8G")
train_list=list(tourney_result.columns)[:-1]
train_1=h2o.H2OFrame(tourney_result)
train_1['result']=train_1['result'].asfactor()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.
Parse progress: |█████████████████████████████████████████████████████████| 100%


H2O_cluster_uptime:,25 mins 54 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 17 days
H2O_cluster_name:,H2O_from_python_fyenne_2smjm1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.814 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [41]:
param = {
      "ntrees" : 2000
    , "max_depth" : 20
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4240
    , "score_tree_interval": 100
    ,"stopping_metric" :"MSE"
    ,"nfolds":8
    ,"fold_assignment":"AUTO"
    ,"keep_cross_validation_predictions" : True
    # ,"booster":"dart"
}

In [ ]:
from h2o.estimators import H2OXGBoostEstimator
model_xgb = H2OXGBoostEstimator(**param)
model_xgb.train(x = train_list, y = 'result', training_frame = train_1)
param={
    "ntrees" : 1060
    , "max_depth" : 20
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4240
    , "score_tree_interval": 100
    , "stopping_metric" :"MSE"
    , "nfolds":8
    , "fold_assignment":"AUTO"
    , "keep_cross_validation_predictions" : True
 }

/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


xgboost Model Build progress: |███████████████████████████████████

In [34]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator  # import gbm estimator
model_gbm = H2OGradientBoostingEstimator(**param)
model_gbm.train(x = train_list, y = 'result', training_frame = train_1)
from h2o.estimators import H2OStackedEnsembleEstimator
stack = H2OStackedEnsembleEstimator(model_id="ensemble11",
                                       training_frame=train_1,
                                       #validation_frame=test,
                                       base_models=[model_xgb.model_id,model_gbm.model_id],metalearner_algorithm="glm")
stack.train(x=train_list, y="result", training_frame=train_1)


/Users/fyenne/miniforge3/envs/siming/lib/python3.8/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


gbm Model Build progress: |███████████████████████████████████████████████| 100%
stackedensemble Model Build progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_900295f736b5e6756810fc249414486 failed with an exception: water.exceptions.H2OIllegalArgumentException: Base models are inconsistent: they use random-seeded k-fold cross-validation but have different seeds.
stacktrace: 
water.exceptions.H2OIllegalArgumentException: Base models are inconsistent: they use random-seeded k-fold cross-validation but have different seeds.
	at hex.ensemble.StackedEnsembleModel.checkAndInheritModelProperties(StackedEnsembleModel.java:527)
	at hex.ensemble.StackedEnsemble$StackedEnsembleDriver.computeImpl(StackedEnsemble.java:335)
	at hex.ModelBuilder$Driver.compute2(ModelBuilder.java:243)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1575)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:974)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1477)
	at jsr166y.ForkJoinWorkerThrea

In [ ]:
test_1=h2o.H2OFrame(test_df)
pred1=model_xgb.predict(test_1)
pred2=model_gbm.predict(test_1)
pred_df1=pred1.as_data_frame()
pred_df2=pred2.as_data_frame()

In [ ]:
ids = pd.read_csv(path + '/MSampleSubmissionStage2.csv').ID.values
values = pred_df2.p1 * 0.7 + pred_df1.p1 * 0.3
# values = pred_df1.p1

final_df = pd.DataFrame(columns=['ID', 'Pred'])
final_df.ID = ids
final_df.Pred = values
final_df.to_csv('submission_men_GBM_worse.csv', columns=['ID','Pred'], index=None)

In [27]:
1+1

2